In [22]:
# %% loading keras LSTM and other libs
from keras.models import Sequential
from keras.layers import Dense, LSTM, RNN
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt
import pickle
import tensorflow as tf 
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


Num GPUs Available:  1


In [23]:
# Example list of documents
file = open('datasets/sample.txt','r')

documents = [
    file.read(50000)
]


In [24]:
# Initialize a TfidfVectorizer object with custom options
vectorizer = TfidfVectorizer(max_features=5000)
tfidf_matrix = vectorizer.fit_transform(documents)

In [25]:
def NL_training_segmentation(data):
    data = vectorizer.transform(data.split(' ')).toarray()
    X = []
    y = []
    for i in range(0,len(data)-1):
        X.append([data[i]])
        y.append(data[i+1])
    return np.array(X),np.array(y)
def Basic_segemntation(data):
    data = vectorizer.transform(data.split(' ')).toarray()
    X = []
    for i in range(0,len(data)):
        X.append([data[i]])
    return np.array(X)

In [26]:
X_train , y_train = NL_training_segmentation(documents[0])
X_test , y_test = NL_training_segmentation(documents[0])
# X_train , y_train , X_test , y_test = np.array(X_train) , np.array(y_train) , np.array(X_test) , np.array(y_test)


print(X_train.shape)

(5190, 1, 1499)


In [39]:
# Creating the neural network model
model = Sequential()

# Adding the layers to the model
model.add(LSTM(1024, input_shape=(1,1499))) # LSTM layer with 128 neurons, input shape of (1, input_size)
# model.add(Dense(64, activation='relu')) # Hidden layer with neurons
# model.add(Dense(64, activation='relu')) # Hidden layer with neurons
# model.add(Dense(64, activation='relu')) # Hidden layer with neurons
# model.add(Dense(64, activation='relu')) # Hidden layer with neurons

model.add(Dense(1499, activation='sigmoid')) # Output layer with 1 neuron

linear_loss ='mean_squared_error'
sigmoid_loss = 'binary_crossentropy'

# Compiling the model
model.compile(loss=sigmoid_loss, optimizer='Adam', metrics=['accuracy'])

2023-04-28 21:44:18.967013: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-28 21:44:18.968005: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-28 21:44:18.968630: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

In [40]:
history = model.fit(X_train, y_train, epochs=10, batch_size=1)
plt.plot(history.history['loss'], label='Training Loss')
plt.legend()
plt.show()

Epoch 1/10


2023-04-28 21:44:20.795473: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-28 21:44:20.796470: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-28 21:44:20.797146: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

3529/5190 [===================>..........] - ETA: 13s - loss: 0.0179 - accuracy: 0.0595

KeyboardInterrupt: 

In [17]:
score = model.evaluate(X_test, y_test, batch_size=1)
print("Accuracy: %.2f%%" % (score[0]*100))
print("Accuracy: %.2f%%" % (score[1]*100))

  94/5190 [..............................] - ETA: 5s - loss: 0.0035 - accuracy: 0.0000e+00     

2023-04-28 21:23:13.721345: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-04-28 21:23:13.723024: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-04-28 21:23:13.723716: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

5190/5190 [==============================] - 5s 929us/step - loss: 0.0030 - accuracy: 0.0114
Accuracy: 0.30%
Accuracy: 1.14%


In [36]:
vocabulary = vectorizer.get_feature_names_out()
prompt = 'yet him'
for i in range(0,10):
    promptx = Basic_segemntation(prompt)
    y_pred = model.predict(np.array([promptx[i+1]]))
    word = vocabulary[np.where(y_pred == y_pred.max())[1][0]]
    prompt += " "+word
print(prompt)



1/1 [==============================] - 0s 15ms/step
yet him for the host who had been in the host who


In [60]:
# Save the trained model as a pickle string.
saved_model = pickle.dumps(model)
# %%
saved_model
# %%
file = open(input("enter name of model:"),"wb")
file.write(saved_model)
# %%

3377979